In [1]:
import cv2
from mtcnn.mtcnn import MTCNN # import MTCNN
from OpenSSL import crypto
from socket import gethostname
import cv2
import base64
import hashlib
import os
import sys


Using TensorFlow backend.


###  Creates signed privacy protected video

In [2]:


detector = MTCNN()#MTCNN initialized
CERT_FILE = "selfsigned.crt"
KEY_FILE = "private.key"
NumberPlateDetectorPath='E:\\Sem7\\IS\\Proj\\PintoPrototype\\haarcascade_russian_plate_number.xml'
cascade = cv2.CascadeClassifier(NumberPlateDetectorPath)#Haar initialized
signaturePath='signature'
storeSignaturePath='out.png'


def find_and_blur(bw, color): 
    # detect al faces

    facesMtcnn = detector.detect_faces(color) #applying mtcnn

     ##########################################################
    for i in range(len(facesMtcnn)):#Mtcnn
        # get coordinates
        x1, y1, width, height = facesMtcnn[i]['box']#getting bounding box of face
        x2, y2 = x1 + width, y1 + height
        roi_color = color[y1:y2, x1:x2]
        # blur the colored image
        blur = cv2.GaussianBlur(roi_color, (101,101), 0)
        # Insert ROI back into image
        color[y1:y2, x1:x2] = blur 

    ##########################################################

    return color

def hash_file(filename):
   """"This function returns the SHA-1 hash
   of the file passed into it"""
   # make a hash object
   h = hashlib.sha1()
   
   # open file for reading in binary mode
   with open(filename,'rb') as file:
       # loop till the end of the file
       chunk = 0
       while chunk != b'':
           # read only 1024 bytes at a time
           chunk = file.read(1024)
           h.update(chunk)
   # return the hex representation of digest
   return h.hexdigest()

def signContent(signImagePath):
    try:
        # Now the real world use case; use certificate to verify signature
        f = open(KEY_FILE)
        pv_buf = f.read()
        f.close()
        priv_key = crypto.load_privatekey(crypto.FILETYPE_PEM, pv_buf)
        
        # # sign and verify PASS
        msg=hash_file(signImagePath)
        sig = crypto.sign(priv_key, msg, 'sha256')#signed the image
        
        open(signaturePath, 'a').close()

        f = open(signaturePath, 'wb')
        f.write(sig)
        f.close()
        
    except:
        print("File path may not be correct")

# turn camera on

video_capture = cv2.VideoCapture(0)


out = cv2.VideoWriter('videoOut.mp4',cv2.VideoWriter_fourcc(*'MJPG'),20.0, (round(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)),round(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while True:
    # get last recorded frame
    _, color = video_capture.read()
    
    # detect the face and blur it
    blur = find_and_blur(color, color)
    # display output
    cv2.imshow('Video', blur)
    out.write(blur)
    # break if q is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# turn camera off        
video_capture.release()
out.release()
# close camera  window
cv2.destroyAllWindows()

signContent("videoOut.mp4")


Instructions for updating:
Colocations handled automatically by placer.


C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: str for data is no longer accepted, use bytes


### Checks forgery in the privacy protected video if any

In [23]:

def detectForgery(received):
    
    try:
        #open certificate
        f = open(CERT_FILE)
        ss_buf = f.read()
        f.close()
        ss_cert = crypto.load_certificate(crypto.FILETYPE_PEM, ss_buf)

        #open the signed content received
        f = open(signaturePath, 'rb')
        sig=f.read()
        f.close()

        # verify the integrity of received image with the certificate and signature
        crypto.verify(ss_cert, sig, hash_file(received), 'sha256')
        print("Authenticated")
    
    except:
        print("Forgery detected")
        
detectForgery("videoOut.mp4")

Authenticated


C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: str for data is no longer accepted, use bytes
